# Step 1 - Data Engineering
***
Import, analysis and data cleansing of CSV data

In [1]:
# Importing Dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Path to data files
measurements_csv = "data_files/hawaii_measurements.csv"
stations_csv = "data_files/hawaii_stations.csv"

In [3]:
# Reading files into Pandas Dataframe
hawaii_meas_df = pd.read_csv(measurements_csv, header="infer",dtype={"station":"str","date":"str","prcp":"float","tobs":"float"} )
hawaii_stat_df = pd.read_csv(stations_csv, header="infer")

In [4]:
# Verifying Data and potential cleaning actions
hawaii_meas_df.describe(include="all")

,station,date,prcp,tobs
count,19550,19550,18103.000000,19550.000000
unique,9,2792,NaN,NaN
top,USC00519281,2012-04-04,NaN,NaN
freq,2772,9,NaN,NaN
mean,NaN,NaN,0.160644,73.097954
std,NaN,NaN,0.468746,4.523527
min,NaN,NaN,0.000000,53.000000
25%,NaN,NaN,0.000000,70.000000
50%,NaN,NaN,0.010000,73.000000
75%,NaN,NaN,0.110000,76.000000


In [5]:
# Verifying Data and potential cleaning actions
hawaii_stat_df.describe(include="all")

,station,name,latitude,longitude,elevation
count,9,9,9.000000,9.000000,9.000000
unique,9,9,NaN,NaN,NaN
top,USC00516128,"HONOLULU OBSERVATORY 702.2, HI US",NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN
mean,NaN,NaN,21.393826,-157.867098,60.977778
std,NaN,NaN,0.086442,0.103873,103.465547
min,NaN,NaN,21.271600,-158.011100,0.900000
25%,NaN,NaN,21.333100,-157.975100,7.000000
50%,NaN,NaN,21.393400,-157.837400,14.600000
75%,NaN,NaN,21.451670,-157.802500,32.900000


In [6]:
# Establishing index for Stations dataframe
if "station" in hawaii_stat_df.columns:
    hawaii_stat_df.set_index("station", inplace=True)
clean_hawaii_stat_df = hawaii_stat_df

In [7]:
# Creating a set of data that finds the average precipitation by station and month
hawaii_meas_avg_df = pd.DataFrame(hawaii_meas_df.loc[:,["station","date","prcp","tobs"]])
hawaii_meas_avg_df["prcp"] = hawaii_meas_avg_df["prcp"].astype(float)
hawaii_meas_avg_df["tobs"] = hawaii_meas_avg_df["tobs"].astype(float)
hawaii_meas_avg_df["date"] = pd.to_datetime(hawaii_meas_avg_df["date"],format="%Y-%m-%d", errors="coerce")
hawaii_meas_avg_df["year"] = hawaii_meas_avg_df["date"].dt.year
hawaii_meas_avg_df["month"] = hawaii_meas_avg_df["date"].dt.month
hawaii_meas_avg_df["day"] = hawaii_meas_avg_df["date"].dt.day
if "station" in hawaii_meas_avg_df.columns:
    hawaii_meas_avg_df["station_month"] = hawaii_meas_avg_df["station"].astype(str, errors="ignore")+"_"+hawaii_meas_avg_df["month"].astype(str)
else:
    hawaii_meas_avg_df["station_month"] = hawaii_meas_avg_df.index.astype(str, errors="ignore")+"_"+hawaii_meas_avg_df["month"].astype(str)
avg_prcp_by_month = hawaii_meas_avg_df.loc[:,["station_month","prcp"]]

In [8]:
# Dropping all NaN values in "prcp" column and creating an averages group by 
# the "station_month" column
avg_prcp_by_month.dropna(inplace=True)
avg_prcp_by_month=avg_prcp_by_month.groupby(["station_month"])["prcp"].mean()


In [9]:
# Replacing all NaN values with the average "prcp" data by station and month
hawaii_meas_avg_df.prcp = round(hawaii_meas_avg_df.prcp.fillna(
    hawaii_meas_avg_df.station_month.map(avg_prcp_by_month).astype(float)),2)

In [10]:
# Evaluating if the "station" column is the index, and establishing "station" 
# as the index if not.
if "station" in hawaii_meas_avg_df.columns:
    hawaii_meas_avg_df.set_index("station", inplace=True)
    print("Index assigned to 'station' column.")
else:
    print("Index has already been assigned to'station' column.")
    
hawaii_meas_df = hawaii_meas_avg_df

Index assigned to 'station' column.


In [11]:
# Saving all of the new columns added to a separate dataframe
attr_hawaii_meas_df = hawaii_meas_avg_df

# Verifying index has not been set and establishing the index for new dataframe
if "station" in attr_hawaii_meas_df.columns:
    attr_hawaii_meas_df.set_index("station", inplace=True)
else:
    ""


In [12]:
# Final cleansing of Measurement data by verifying and establishing index of "station" column
if "station" in hawaii_meas_df.columns:
    clean_hawaii_meas_df = hawaii_meas_df.loc[:,["station","date","prcp","tobs"]]
    hawaii_meas_df.set_index("station", inplace=True)
    print("Index assigned to 'station' column.")
else:
    clean_hawaii_meas_df = hawaii_meas_df.loc[:,["date","prcp","tobs"]]
    print("Index has already been assigned to'station' column.")

Index has already been assigned to'station' column.


In [13]:
# Outputting cleaned versions of each dataframe
clean_hawaii_meas_df.to_csv(measurements_csv.replace("data_files/","data_files/CLEAN_"))
clean_hawaii_stat_df.to_csv(stations_csv.replace("data_files/","data_files/CLEAN_"))

In [14]:
# Saving output of additional fields for reference or analysis.
attr_hawaii_meas_df.to_csv("data_files/measurements_with_additional_attributes.csv")